In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
!pip install openai==0.28.0
!pip install huggingface_hub
!pip install groq
!pip instrall requests

ERROR: unknown command "instrall" - maybe you meant "install"


In [70]:
import os
from groq import Groq
from huggingface_hub import InferenceClient
import pandas as pd
import json
from google.colab import userdata

In [71]:
file_path = '/content/drive/My Drive/data.csv'
data = pd.read_csv(file_path)
case_row = data.iloc[497]
full_text = case_row['text']
case_summary = full_text
accusation = full_text
testimony_prompt = full_text

In [72]:
import os
import google.colab
from huggingface_hub import InferenceClient, login, HfFolder
from transformers import AutoTokenizer


hf_token = os.getenv("HF_TOKEN")


if hf_token and not HfFolder.get_token():
    login(token=hf_token)


hf_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B", token=hf_token) #tokenizer

#truncate the prompt
def truncate_prompt(prompt, max_input_tokens=3900):
    input_ids = hf_tokenizer(prompt, return_tensors="pt").input_ids[0]
    if len(input_ids) > max_input_tokens:
        prompt = hf_tokenizer.decode(input_ids[:max_input_tokens], skip_special_tokens=True)
    return prompt


In [73]:
import os
import time
import requests
import google.colab
class Agent:
    def __init__(self, name, role, llm_type):
        self.name = name
        self.role = role
        self.llm_type = llm_type.lower()
        self.memory = [] #memory for each agent is stored

        #initialization of hugging face client
        if self.llm_type == "hf":
            self.hf_client = InferenceClient("google/flan-t5-base") #google flan or 'microsoft/Phi-3-mini-4k-instruct'

        elif self.llm_type == 'groq':
          self.groq_api_key = os.getenv("GROQ_API")
          self.groq_url = "https://api.groq.com/openai/v1/chat/completions"


    def add_to_memory(self, prompt, response):
      self.memory.append({"prompt": prompt, "response": response})

    def get_context(self, window_size=2): #remembers 2 converstaions
      if not self.memory:
        return ""  # is empty if there is no memory yet
      context = ""
      for item in self.memory[-window_size:]:
          context += f"User: {item['prompt']}\n{self.name}: {item['response']}\n"
      return context



    def generate_response(self, prompt):
        context = self.get_context()
        full_prompt = context + f"User: {prompt}\n{self.name}:" #total prompt including memory

        if self.llm_type == 'hf':
          response = self.query_huggingface(full_prompt)

        elif self.llm_type == 'groq':
          response = self.query_groq(full_prompt)
          time.sleep(15)


        else:
            response = f"{self.name} ({self.role}): Unsupported LLM type."

        self.add_to_memory(prompt, response)
        return response





    def query_huggingface(self, prompt):
      try:

        prompt = truncate_prompt(prompt, max_input_tokens=2048) #truncate the prompt
        return self.hf_client.text_generation(prompt, max_new_tokens=300)
      except Exception as e:

        #if failed,try with a smaller truncated prompt
        try:
            return "[HuggingFace Error - retrying with shorter prompt] " + \
                   self.hf_client.text_generation(prompt[:300], max_new_tokens=30)
        except Exception as e2:
            return f"[HF Final Error] {str(e2)}"




    def query_groq(self, prompt):
      try:

          if len(prompt) > 7000 :
            prompt = prompt[-7000:] #take 7000
          headers = {
            'Authorization': f'Bearer {self.groq_api_key}',
            'Content-Type': 'application/json'
          }
          messages = [
            {"role": "system", "content": f"You are {self.name}, a {self.role} in a courtroom. Respond accordingly."},
            {"role": "user", "content": prompt}
          ]
          payload = {
              "model": "llama-3.3-70b-versatile",
              "messages": messages,
              "max_tokens": 300,
              "temperature": 0.3,
              "top_p" : 0.8,
              "stream" : False
          }
          response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload)
          response_json = response.json()
          if response.status_code == 200:
            return response_json['choices'][0]['message']['content'].strip()

          elif response.status_code == 429:  # rate limit

            if "Retry-After" in response.headers:
                wait_time = float(response.headers["Retry-After"])
            else:
                wait_time = 6  # fallback
            print(f"[Rate Limit] Waiting {wait_time} seconds...")
            time.sleep(wait_time)
            return self.query_groq(prompt)

          else:
            return f"[Groq Error] {response_json.get('error', 'Unknown error')}"
      except Exception as e:
          return f"[Groq Error] {str(e)}"


In [74]:
class CourtroomAgents:
    def __init__(self):
        self.agents = {}

    def add_agent(self, name, role, llm_type):
        agent = Agent(name, role, llm_type)
        self.agents[role.lower()] = agent

    def get_agent(self, role):
        return self.agents.get(role.lower(), None)

    def simulate_opening_statements(self, case_summary, accusation):
        print(" Trial Opening Statements")

        judge = self.get_agent('judge')
        prosecutor = self.get_agent('prosecutor')
        defense = self.get_agent('defense lawyer')
        witness = self.get_agent('witness')
        defendent = self.get_agent('defendent')
        plaintiff = self.get_agent('plaintiff')
        # Judge opens court
        judge_prompt = f"You are a judge. Open the courtroom and summarize the case precisely without showing any bias: {case_summary}"
        casetype =judge.generate_response(f"you have read the summary of the case : {case_summary} now tell what type of case it is(answer should be of the form (civil case) or (criminal case) in one word)")
        print(f"============== {casetype} ====================")
        print(f" Judge {judge.name}:")
        print(judge.generate_response(judge_prompt))

        # Prosecutor gives opening statement
        prosecutor_prompt = f"You are a prosecutor. Present the accusations against the defendant correctly and shortly.Be confident and aggressive : {accusation}"
        print(f"\n Prosecutor {prosecutor.name}:")
        print(prosecutor.generate_response(prosecutor_prompt))

        #plaintiff gives his side
        if "civil" in casetype.lower():
          plaintiff_prompt = f"You are the plaintiff. Express your distress to the judge: {case_summary}"
          print(f"\n Plaintiff {plaintiff.name}:")
          print(plaintiff.generate_response(plaintiff_prompt))


          # Defense gives opening statement
          defense_prompt = f"You are a defense lawyer. Begin your defense for the accused.Be calculative and answer more accurately. Accusation: {accusation}"
          print(f"\n Defense Lawyer {defense.name}:")
          print(defense.generate_response(defense_prompt))

    def simulate_witness_testimony(self, witness_name, role, testimony_prompt):
        witness = self.get_agent("witness")
        print(f"\n {witness_name} ({role}) is called to testify.")
        testimony = witness.generate_response(f"based on the following text produce a testimony for the witness:\n\n'{testimony_prompt}'") #asking llm to observe a testimony in the text

        response = witness.generate_response(f"you need to Testify as a {role} on the following: {testimony}") #witness gives his testification
        print(f"\n{witness_name} testifies: {response}")

    def simulate_cross_examination(self, questioning_party, questioned_party, testimony):

      questioning_agent = self.get_agent(questioning_party)
      questioned_agent = self.get_agent(questioned_party)

      # Ask LLM to generate a cross-exam question based on the testimony
      question_prompt = f"Based on the following testimony, generate a critical cross-examination question:\n\n'{testimony}'"
      question = questioning_agent.generate_response(question_prompt)

      print(f"\n Cross-Examination by {questioning_party} of {questioned_party}")
      print(f"{questioning_party}: {question}")

      # questioned party responds to the question
      response = questioned_agent.generate_response(f"{questioning_party} asks: {question} generate a precise response")
      print(f"{questioned_party}: {response}")

    def simulate_closing_statements(self):
        jury = self.get_agent('jury')
        print("\n Closing Statements")

        judge = self.get_agent('judge')
        prosecutor = self.get_agent('prosecutor')
        defense = self.get_agent('defense lawyer')

        # Prosecutor closing statement
        prosecutor_prompt = "Make your closing argument and request the judge to take appropriate measures."
        print(f"\n Prosecutor {prosecutor.name}:")
        print(prosecutor.generate_response(prosecutor_prompt))

        # Defense closing statement
        defense_prompt = "Make your closing argument and request the judge to take appropriate measures."
        print(f"\n Defense Lawyer {defense.name}:")
        print(defense.generate_response(defense_prompt))

        # Judge gives the verdict
        judge_prompt = "Give the verdict on the case considering each and every detail."
        print(f"\n Judge {judge.name}:")
        print(judge.generate_response(judge_prompt))

        #jury gives the verdict
        jury_prompt = "learn about the whole case and inspect all details and tell whether the justice is done or not"
        print(f"\n Jury gives the statement: \n")
        print(jury.generate_response(jury_prompt))


In [ ]:
court = CourtroomAgents()

#creating agents
court.add_agent("Justice Ray", "Judge", "groq")
court.add_agent("Ava", "Prosecutor", "groq")
court.add_agent("Leo", "Defense Lawyer", "groq")
court.add_agent("Bob", "witness", "groq")
court.add_agent("defendent","defendent","hf")
court.add_agent("jury","jury","groq")
court.add_agent("plaintiff","plaintiff","groq")

#run the process
court.simulate_opening_statements(case_summary, accusation)
court.simulate_witness_testimony("witness","witness",testimony_prompt)
court.simulate_cross_examination("Defense Lawyer","Prosecutor",testimony_prompt)
court.simulate_cross_examination("Prosecutor","witness",testimony_prompt)
court.simulate_cross_examination("Prosecutor","Defense Lawyer",testimony_prompt)
court.simulate_cross_examination("Prosecutor","Defendent",accusation)
court.simulate_closing_statements()

 Trial Opening Statements
[Rate Limit] Waiting 744.0 seconds...
